# Cleaned vs Dirty

<img src="https://www.kaggle.com/competitions/15282/images/header">

Bu projemizde tabakların temiz mi yoksa kirli mi oldugunu bulan bir model gelistirecegiz.

<a href="https://www.kaggle.com/competitions/platesv2/data">Dataya Buradan Erişebilirsiniz</a>

In [1]:
import cv2
import pandas as pd
import os

import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split

In [2]:
img_path="train/"
labels=os.listdir("train")

In [3]:
labels

['cleaned', 'dirty']

In [4]:
#Dosyadan Resim okuma
img_list=[]
label_list=[]
for label in labels:
    for img_file in os.listdir(img_path+label):       #Klasörün içindeki alt klasörleri söyler
        img_list.append(img_path+label+"/"+img_file)  #apppend dizinin sonuna eklme yapıyor
        label_list.append(label)

In [5]:
df=pd.DataFrame({"img":img_list,"label":label_list})

In [6]:
df.head()

,img,label
0,train/cleaned/0000.jpg,cleaned
1,train/cleaned/0001.jpg,cleaned
2,train/cleaned/0002.jpg,cleaned
3,train/cleaned/0003.jpg,cleaned
4,train/cleaned/0004.jpg,cleaned


In [7]:
d={'cleaned':0, 'dirty':1}

In [8]:
df["encode_label"]=df["label"].map(d)

In [9]:
df

,img,label,encode_label
0,train/cleaned/0000.jpg,cleaned,0
1,train/cleaned/0001.jpg,cleaned,0
2,train/cleaned/0002.jpg,cleaned,0
3,train/cleaned/0003.jpg,cleaned,0
4,train/cleaned/0004.jpg,cleaned,0
...,...,...,...
137,train/dirty/0596.jpg,dirty,1
138,train/dirty/0703.jpg,dirty,1
139,train/dirty/0707.jpg,dirty,1
140,train/dirty/0710.jpg,dirty,1


In [10]:
def process_image(img):
        img=cv2.imread(img)
        img=cv2.resize(img,(70,70))
        img=img/255
        x.append(img)
        return img

In [11]:
x=[]
for img in df["img"]:
    process_image(img)    

In [12]:
x = np.array(x)

In [13]:
y=df["encode_label"]

In [14]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.20,random_state=42)

In [15]:
from keras.models import Sequential

In [16]:
from keras.layers import Conv2D,Dense,Flatten, Input, MaxPooling2D, Dropout, BatchNormalization, Reshape

In [17]:
model=Sequential()
model.add(Input(shape=(70,70,3)))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.25)) # Burada 'rate' parametresini belirtiyoruz
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(128))
model.add(Dense(1, activation='sigmoid')) 
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [18]:
history=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=15,verbose=1)

Epoch 1/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 185ms/step - accuracy: 0.5869 - loss: 3.5911 - val_accuracy: 0.4828 - val_loss: 1.0552
Epoch 2/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.8298 - loss: 4.1016 - val_accuracy: 0.6552 - val_loss: 0.5760
Epoch 3/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.8884 - loss: 1.9631 - val_accuracy: 0.5517 - val_loss: 1.5301
Epoch 4/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.9352 - loss: 1.0585 - val_accuracy: 0.5517 - val_loss: 1.0224
Epoch 5/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.8665 - loss: 1.9331 - val_accuracy: 0.7931 - val_loss: 0.5508
Epoch 6/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.9706 - loss: 0.3567 - val_accuracy: 0.4483 - val_loss: 1.1188
Epoch 7/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.9173 - loss: 0.4591 - val_accuracy: 0.6207 - val_loss: 0.6909
Epoch 8/15
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.9313 - loss: 0.3697 - val_accuracy: 0.5517 - val_loss: 1.8095

## VGG16

In [19]:
from keras.models import Sequential
from keras.layers import Conv2D,Dense,Flatten, Input, MaxPooling2D, Dropout, BatchNormalization, Reshape
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Tek satırda resimlerin tamamını okumaya yarıyor

In [20]:
data_dir="train"
img_width,img_height=70,70

train_datagen=ImageDataGenerator(rescale=1/255, validation_split=0.20)

train_datagenerator=train_datagen.flow_from_directory(directory=data_dir,target_size=(img_width,img_height),
                                class_mode="binary", subset="training")

test_datagen=ImageDataGenerator( rescale=1/255)
test_datagenerator=train_datagen.flow_from_directory(directory=data_dir,target_size=(img_width,img_height),
                                class_mode="binary", subset="validation")

base_model=VGG16(weights="imagenet", input_shape=(img_width,img_height,3),include_top=False)
model=Sequential()

model.add(base_model)
for layer in base_model.layers:
    layer.trainable=False

model.add(Flatten())
model.add(Dense(1024,activation="relu")) # Dense nöronları birbirine bağlar
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.fit(train_datagenerator,epochs=10,validation_data=test_datagenerator)

Found 114 images belonging to 2 classes.
Found 28 images belonging to 2 classes.
Epoch 1/10


C:\Users\erkan\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 547ms/step - accuracy: 0.4922 - loss: 1.4541 - val_accuracy: 0.5000 - val_loss: 1.3113
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 371ms/step - accuracy: 0.7799 - loss: 0.6093 - val_accuracy: 0.5714 - val_loss: 0.8205
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 341ms/step - accuracy: 0.8646 - loss: 0.2636 - val_accuracy: 0.5357 - val_loss: 1.0707
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 329ms/step - accuracy: 0.8992 - loss: 0.1904 - val_accuracy: 0.6071 - val_loss: 0.7321
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 307ms/step - accuracy: 0.9625 - loss: 0.1294 - val_accuracy: 0.6071 - val_loss: 0.7512
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 400ms/step - accuracy: 0.9595 - loss: 0.0880 - val_accuracy: 0.6071 - val_loss: 0.9291
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 379ms/step - accuracy: 0.9748 - loss: 0.0661 - val_accuracy: 0.5714 - val_loss: 1.1027
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 431ms/step - accuracy: 0.9571 - loss: 0.0880 - val_accuracy: 0.6071 - val_loss: 0.9418
Epo

In [21]:
model.save("clean.h5")

Sonuc olarak modelimizi hem CNN hem de transfer learning kullanarak egittik. Ancak veri seti oldukca azdi. Bu da modelimizin dogruluk oranini olumsuz yonde etkiledi.